In [1]:
!python -V

Python 3.11.3


In [2]:
import pandas as pd
import seaborn as sns
import pickle
import matplotlib.pyplot as plt

from pathlib import Path

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [4]:
import mlflow
TRACKING_SERVER_HOST = "0.0.0.0"
MLFLOW_EXPERIMENT_NAME = 'nyc-taxi-experiment'

# Specify Public URL of EC2 instance where the MLflow tracking server is running

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:8080") 
print(f"Tracking Server URI: '{mlflow.get_tracking_uri()}'")

mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

Tracking Server URI: 'http://0.0.0.0:8080'


2023/08/19 11:58:06 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://upskills-landing-zone/mlflow/1', creation_time=1692417487886, experiment_id='1', last_update_time=1692417487886, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [5]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df['duration'].dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df.loc[:,categorical] = df[categorical].astype(str)
    
    return df

In [6]:
data_path = Path("__file__").resolve().parents[1] / 'data'
train_data_path = (data_path / 'yellow_tripdata_2022-01.parquet').as_posix()
test_data_path = (data_path / 'yellow_tripdata_2022-02.parquet').as_posix()

In [7]:
df_train = read_dataframe(train_data_path)
df_val = read_dataframe(test_data_path)

In [8]:
len(df_train), len(df_val)

(2421440, 2918187)

In [9]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [10]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [12]:
data_desc = f"Description: train data is from 2022-01 and test-data is from 2022-02"
with mlflow.start_run(run_name='logistic-regression',
                      tags={"mlflow.note.content": data_desc}):

    mlflow.set_tag("developer", "haquan")
    # log train & test data path
    mlflow.log_param("train-data-path", train_data_path)
    mlflow.log_param("valid-data-path", test_data_path)

    alpha = 0.1
    # log parameter 
    mlflow.log_param("alpha", alpha) 
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    # This is to log the preprocessor as artifact
    """
    with open('models/preprocessor.bin', 'wb') as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact(local_path="models/preprocessor.bin", artifact_path="preprocessor")
 
    
    # This is to log model & register
    mlflow.sklearn.log_model(sk_model=lr,
                             artifact_path="models", # path on the mlflow server's artifact
                             registered_model_name="linear-regression-baseline"
    )
    """

In [13]:
import xgboost as xgb
# Hyperopt: based on Baysian method
# fmin: minium output, hp: search space
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [14]:
# parse the data to XGB data type
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [15]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [16]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)), #q-uniform method to find max_depth
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}
# f-min function: try to minimize the "objective" function
# this is to perform 1 exp as max_evals=1
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=1,
    trials=Trials()
)

[12:05:49] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:13.01694                         
[1]	validation-rmse:10.56719                         
[2]	validation-rmse:8.83385                          
[3]	validation-rmse:7.63009                          
[4]	validation-rmse:6.80959                          
[5]	validation-rmse:6.25576                          
[6]	validation-rmse:5.88571                          
[7]	validation-rmse:5.63819                          
[8]	validation-rmse:5.47092                          
[9]	validation-rmse:5.35610                          
[10]	validation-rmse:5.27563                         
[11]	validation-rmse:5.21932                         
[12]	validation-rmse:5.17806                         
[13]	validation-rmse:5.14850                         
[14]	validation-rmse:5.12554 

In [18]:
mlflow.xgboost.autolog(disable=True)

In [21]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[11:40:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95634
[2]	validation-rmse:16.59114
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32800
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47583
[10]	validation-rmse:9.96483
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12620
[13]	validation-rmse:8.78522
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23332
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65209
[19]	validation-rmse:7.50743
[20]	validation-rmse:7.38235
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rmse:6.91001
[27]	valid

In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
